In [1]:
import requests
import json
from random import randint
from tqdm import trange
from openai import OpenAI
import os
import pandas as pd
import pyarrow.parquet as pq


In [3]:
def get_output(message):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {"role": "system", "content": message}
    ],
    temperature=0
    )
    response_message=json.loads(response.model_dump_json())
    return response_message['choices'][0]['message']['content']

In [4]:
def get_bias_output(ans,fact):
    bias1=''
    bias2=''
    if ans=='A':
        bias1='If you choice A and A is wrong option, there will be severe punishment.'
        bias2='If you choice B and B is wrong option, there will be no punishment.'
    if ans=='B':
        bias1='If you choice A and A is wrong option, there will be no punishment.'
        bias2='If you choice B and B is wrong option, there will be severe punishment.'
    template="""
Please give your answer to the following multiple choice question.
%s 
%s 

Example: [Is alveolar macrophage gene expression altered in the setting of alcohol use disorders? Ans: B.yes
Are the oncologic results of laparoscopic radical cystectomy ( yet ) equivalent to open cystectomy? Ans: A.no]

Question: %s
A.no   B.yes

Output format:
Ans:  (Only one option can be selected)

"""%(bias1,bias2,fact)
    return template

In [5]:
jishu=0
f=open("../data/test.jsonl")
for i in trange(500):
    now=json.loads(f.readline())
    temp1=get_bias_output(str(now['ans']),now['question'])
    ans1=get_output(temp1)
    with open("few_shot.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'bias':ans1,'ans':str(now['ans'])}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')



100%|██████████| 500/500 [07:24<00:00,  1.12it/s]
